# Hankel covariance matrix completion 
- $ H_{k,l} = (I_k \otimes C) H^{xx}_{k,l} (I_l \otimes C)^\top $
- $ H^{xx}_{k,l} = \left[\begin{array}{llll} A \Pi & A^2 \Pi & \ldots & A^l \Pi\\ A^2 \Pi & A^3 \Pi & \ldots & A^{l+1} \Pi\\ \vdots & \vdots & \ddots & \vdots \\ A^{k} \Pi & A^{k+1} \Pi & \ldots & A^{k+l-1} \Pi \end{array} \right] $
- each block of the Hankel cov matrix $H_{k,l}$ exhibits the same structure of missing entries as does cov($y$.
- We here first assume $A,\Pi$ to be known, and apply Srini's idea of joint gradient descent on the whole $C$ to $H_{k,l}$. Then we learn $A, \Pi$ with the other parameters fixed

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
from scipy.io import loadmat
from scipy.optimize import fmin_bfgs, check_grad
import glob, os

os.chdir('../core')
import stitching_ssid as ssid

import pprint

In [ ]:
p,n = 9,3
k,l = n+1,n+1

# create subpopulations
sub_pops = (np.arange(0,p//2+1), np.arange(p//2,p))
print('sub_pops', sub_pops)
obs_idx, idx_grp, overlaps, overlap_grp, idx_overlap, Om, Ovw, Ovc = \
    ssid.get_subpop_stats(sub_pops, p, verbose=True)

for rep in range(20):


    C_true      = np.random.normal(size=(p,n))
    A_true      = np.random.normal(size=(n,n)) #np.diag(np.linspace(0.5, 0.9, n)) #
    
    U,S,V = np.linalg.svd(A_true)
    A_true = U.dot(np.diag(np.linspace(0.5, 0.9, n))).dot(V)

    #B_true      = np.random.normal(size=(n,n))/np.sqrt(n)    
    #Pi_true     = B_true.dot(B_true.T) #np.eye(n) 
    Pi_true = sp.stats.wishart(n, np.eye(n)).rvs()/n
    B_true = np.linalg.cholesky(Pi_true)

    Qs = ssid.comp_model_covariances({'A': A_true, 'Pi': Pi_true, 'C': C_true}, k+l, Om)
    Qs_full = ssid.comp_model_covariances({'A': A_true, 'Pi': Pi_true, 'C': C_true}, k+l)


    
    A_0  = np.diag(np.random.uniform(low=0.7, high=0.8, size=n))
    B_0  = np.random.normal(size=(n,n))
    Pi_0 = B_0.dot(B_0.T)
    C_0  = np.random.normal(size=(p,n))
    pars_0 = np.hstack((A_0.reshape(n*n,),
                        B_0.reshape(n*n,),
                        C_0.reshape(p*n,)))
    H_0 = ssid.yy_Hankel_cov_mat( C_0,A_0,Pi_0,k,l,~Om)
    
    f_i, g_i = ssid.l2_setup(k,l,n,Qs,Om,idx_grp,obs_idx)
    
    print('difference in gradient to finite-differencing value:', check_grad(f_i, g_i, pars_0))    
    
    pars_est = fmin_bfgs(f_i, pars_0, fprime=g_i, gtol=1e-20)    
    A_est = pars_est[:n*n].reshape(n,n)
    B_est = pars_est[n*n:2*n*n].reshape(n,n)
    Pi_est = B_est.dot(B_est.T)
    C_est = pars_est[-p*n:].reshape(p,n)
    
    pars_init = {'A': A_0, 'C': C_0, 'Pi': Pi_0, 'B': B_0}
    pars_est  = {'A': A_est, 'C': C_est, 'Pi': Pi_est, 'B': B_est}
    pars_true = {'A': A_true, 'C': C_true, 'Pi': Pi_true, 'B': B_true}
    ssid.plot_outputs_l2_gradient_test(pars_true, pars_init, pars_est, k, l, Qs, 
                                       Qs_full, Om, Ovc, Ovw, f_i, g_i, if_flip = True)
    
    print('singular values of partial observability matrix for first subpop \n')
    _,s,_ = np.linalg.svd(ssid.observability_mat((A_true, C_true[sub_pops[0],:]), n))
    print(s)

    print('\n singular values of partial observability matrix for second subpop \n')
    _,s,_ = np.linalg.svd(ssid.observability_mat((A_true, C_true[sub_pops[1],:]), n))
    print(s)

    print('\n singular values of (noise) reachability matrix \n')
    _,s,_ = np.linalg.svd(ssid.observability_mat((A_true, B_true), n))
    print(s)

    

# Large(r) systems tests

In [ ]:
p,n = 100,10
k,l = n+1,n+1

# create subpopulations
sub_pops = (np.arange(0,90), np.arange(10,p))
print('sub_pops', sub_pops)
obs_idx, idx_grp, overlaps, overlap_grp, idx_overlap, Om, Ovw, Ovc = \
    ssid.get_subpop_stats(sub_pops, p, verbose=True)

for rep in range(10):
    
    C_true      = np.random.normal(size=(p,n))
    A_true      = np.diag(np.linspace(0.5, 0.9, n)) #

    #B_true      = np.random.normal(size=(n,n))/np.sqrt(n)    
    #Pi_true     = B_true.dot(B_true.T) #np.eye(n) 
    Pi_true = sp.stats.wishart(n, np.eye(n)).rvs()/n
    B_true = np.linalg.cholesky(Pi_true)
    
    Qs = ssid.comp_model_covariances({'A': A_true, 'Pi': Pi_true, 'C': C_true}, k+l, Om)
    Qs_full = ssid.comp_model_covariances({'A': A_true, 'Pi': Pi_true, 'C': C_true}, k+l)
            
    A_0  = np.diag(np.random.uniform(low=0.7, high=0.8, size=n))
    B_0  = np.random.normal(size=(n,n))
    Pi_0 = B_0.dot(B_0.T)
    C_0  = np.random.normal(size=(p,n))
    pars_0 = np.hstack((A_0.reshape(n*n,),
                        B_0.reshape(n*n,),
                        C_0.reshape(p*n,)))
    H_0 = ssid.yy_Hankel_cov_mat( C_0,A_0,Pi_0,k,l,~Om)
    
    def f_i(pars):                        
        A,B,C = pars[:n*n].reshape(n,n), pars[n*n:2*n*n].reshape(n,n), pars[-p*n:].reshape(p,n)
        Pi = B.dot(B.T)
        return ssid.f_l2_Hankel(C,A,Pi,k,l,Qs, Om)*np.sum(Om)*(k*l)
    
    def g_i(pars):        

        A,B,C = pars[:n*n], pars[(n*n):(2*n*n)], pars[-p*n:]
        return ssid.g_l2_Hankel(C,A,B,k,l,Qs,Om,idx_grp, obs_idx)
    
    print('difference in gradient to finite-differencing value:', check_grad(f_i, g_i, pars_0))    
    
    pars_est = fmin_bfgs(f_i, pars_0, fprime=g_i, gtol=1e-20)    
    A_est = pars_est[:n*n].reshape(n,n)
    B_est = pars_est[n*n:2*n*n].reshape(n,n)
    Pi_est = B_est.dot(B_est.T)
    C_est = pars_est[-p*n:].reshape(p,n)
    
    pars_init = {'A': A_0, 'C': C_0, 'Pi': Pi_0, 'B': B_0}
    pars_est  = {'A': A_est, 'C': C_est, 'Pi': Pi_est, 'B': B_est}
    pars_true = {'A': A_true, 'C': C_true, 'Pi': Pi_true, 'B': B_true}
    ssid.plot_outputs_l2_gradient_test(pars_true, pars_init, pars_est, k, l, Qs, 
                                       Qs_full, Om, Ovc, Ovw, f_i, g_i, if_flip = True)
    